# Massaging Gaussian Mixture Model 

jtm98 
Created: 10.29.24

$$ X_i \sim r N (\mu,\sigma^2) + (1-r) N (\beta*\mu,\sigma^2) $$

- $X_i$ is cell expression 
- $r$ is a mixture proportion of label $Z_i$
- $Z_i \in {0,1} $ and represents whether a cell had normal (0) or decreased (1)  expression 
- $\mu$ is the mean of individuals w/normal expression
- $\beta$ is the enhancer coefficient 


In [3]:
import os
import numpy as np 
from cmdstanpy import cmdstan_path, CmdStanModel

In [ ]:
mixture ="""

// Assessing if beta parameter is inferable 
data {
int<lower=1> N; // number of samples
array[N] real X; // expression in the sample 
}

parameters {
real<lower=0,upper=1> r; // mixture proportion of samples 
real<lower=0.0> sigma; // standard deviation 
real mu; // mean of individuals w/ normal expression
real beta; // enhancer coefficient
}

model {
r ~ beta(2,2); // mixture proportion based on natural scale 
mu ~ normal(0,10); 
beta ~ beta(2,2); // enhancer coefificent should be [0,1]
sigma ~ inv_gamma(1,1); 
for(i in 1:N)
target+= log_sum_exp(log(r)+normal_lpdf(X[i]|mu,sigma),
log(1-r)+normal_lpdf(X[i]|beta*mu,sigma));
}

generated quantities {
array[N] real<lower=0,upper=1> PZi;
for(i in 1:N) {
real LP0=log(r)+normal_lpdf(X[i]|mu,sigma);
real LP1=log(1-r)+normal_lpdf(X[i]|beta*mu,sigma);
PZi[i]=exp(LP0-log_sum_exp(LP0,LP1));
}
}
"""
! mkdir mixture
with open('mixture/mixture.stan', 'w') as f: f.write(mixture) 
! cat mixture/mixture.stan

In [3]:
mixture = CmdStanModel(stan_file='mixture/mixture.stan')

21:55:57 - cmdstanpy - INFO - compiling stan file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture.stan to exe file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture
21:56:06 - cmdstanpy - INFO - compiled model executable: /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture


In [5]:

pos_expression = 100
neg_expression = 90 
true_beta = neg_expression / pos_expression # true beta value

# Simulate random data 
pos = np.random.normal(100,0.3,50)
neg = np.random.normal(90,0.3,50)
data = {'X' : np.concatenate([pos,neg]), 'N' : 100}
inits = { 'mu': 1, 'sigma': 0.3, 'r': 0.5, 'beta': .5 }

In [6]:
mixture_fit = mixture.sample(data=data,thin=1,iter_sampling=1000,iter_warmup=50,inits=inits,seed=11051999)
mixture_fit.summary()['Mean']

21:56:36 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

21:56:37 - cmdstanpy - INFO - CmdStan done processing.
21:56:37 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: beta_lpdf: Random variable is -16.1502, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.36777, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.0755, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.02664, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.01178, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.00059, but must be in the interval [0, 1] (in 'mixture.stan', line 19, column 0 to column 17)
	Exception: beta_lpdf: Random variable is 1.05531, b

lp__       -382.865000
r             0.940178
sigma         9.557130
mu           89.785900
beta          0.709097
               ...    
PZi[96]       0.933019
PZi[97]       0.932819
PZi[98]       0.934001
PZi[99]       0.930330
PZi[100]      0.931962
Name: Mean, Length: 105, dtype: float64

In [8]:
estimated_beta = mixture_fit.summary().loc['beta']['Mean']
print('Estimated Beta:',estimated_beta)
print('True Beta:',true_beta)

Estimated Beta: 0.709097
True Beta: 0.9


# Hierarchical Gaussian Mixture Model with Latent Potency

In [ ]:
stan_code ="""
data {
  int<lower=0> N_control;                  // Number of control samples
  int<lower=0> N_experiment;               // Number of experimental samples
  array[N_control] real y_control;         // Expression levels in control samples
  array[N_experiment] real y_experiment;   // Expression levels in experimental samples
  array[N_experiment] int<lower=1, upper=5> guide; // Guide index for each experiment sample (1 to 5)
  array[N_experiment] real enhancer;       // Enhancer index for each experiment sample
}

transformed data {
  real<lower=0> mu_control_observed = mean(y_control);            // Empirical mean of y_control
  real<lower=0> sigma_control_observed = sd(y_control);           // Empirical standard deviation of y_control
  real<lower=0> mu_experiment_observed = mean(y_experiment);      // Empirical mean of y_experiment
  real<lower=0> sigma_experiment_observed = sd(y_experiment);     // Empirical standard deviation of y_experiment
  real<lower=0> beta_observed = mu_experiment_observed / mu_control_observed ;   // Empirical effect size 
}

parameters {
  real<lower=0> mu_control;                // Mean expression in control condition
  real<lower=0, upper=1> B;                // Effect size for potent guides
  real<lower=0> sigma_control;             // Standard deviation in control condition
  real<lower=0> sigma_experiment;          // Standard deviation in experiment condition
  array[5] real<lower=0, upper=1> potency; // Potency for each guide (0 to 1)
}

transformed parameters {
  real mu_experiment;                      // Single mu_experiment for all experimental samples
  
  // Mean expression in potent experimental samples
  mu_experiment = B * mu_control;
}

model {
  // Priors based on provided initial values
  mu_control ~ normal(mu_control_observed, sigma_control_observed);        // Prior for control mean centered around 22,000
  B ~ normal(beta_observed, 0.25);                          // Prior for effect size B around 0.5
  sigma_control ~ cauchy(sigma_control_observed, 2);      // Prior for control standard deviation centered around 4,000
  sigma_experiment ~ cauchy(sigma_experiment_observed, 2);   // Prior for experimental standard deviation centered around 3,700
  potency ~ beta(0.5, 0.5);                    // Prior for potency of each guide, allowing flexibility

  // Likelihoods
  y_control ~ normal(mu_control, sigma_control); // Control samples from control distribution

  // Experimental samples based on guide potency
  for (n in 1:N_experiment) {
    real mean_expr = (1 - potency[guide[n]]) * mu_control + potency[guide[n]] * mu_experiment;
    y_experiment[n] ~ normal(mean_expr, sigma_experiment);
  }
}
"""
name = 'model2'
! mkdir {name}
with open('%s/%s.stan'%(name,name), 'w') as f: f.write(stan_code) 
! cat {name}/{name}.stan

mkdir: model2: File exists

data {
  int<lower=0> N_control;                  // Number of control samples
  int<lower=0> N_experiment;               // Number of experimental samples
  array[N_control] real y_control;         // Expression levels in control samples
  array[N_experiment] real y_experiment;   // Expression levels in experimental samples
  array[N_experiment] int<lower=1, upper=5> guide; // Guide index for each experiment sample (1 to 5)
}

transformed data {
  real<lower=0> mu_control_observed = mean(y_control);            // Empirical mean of y_control
  real<lower=0> sigma_control_observed = sd(y_control);           // Empirical standard deviation of y_control
  real<lower=0> mu_experiment_observed = mean(y_experiment);      // Empirical mean of y_experiment
  real<lower=0> sigma_experiment_observed = sd(y_experiment);     // Empirical standard deviation of y_experiment
  real<lower=0> beta_observed = mu_experiment_observed / mu_control_observed ;   // Empirical effec

In [109]:
model = CmdStanModel(stan_file='%s/%s.stan'%(name,name))

23:16:13 - cmdstanpy - INFO - compiling stan file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model2/model2.stan to exe file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model2/model2
23:16:23 - cmdstanpy - INFO - compiled model executable: /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model2/model2


In [110]:
import pandas as pd 

final_data = pd.read_csv("final/data1.tsv",sep='\t')
geneID, enhancerID = 1,1
data_subset = final_data[final_data.geneID == geneID]

control_samples = data_subset[data_subset.enhancerID == -1 ]
N_control = control_samples.shape[0]

# Calculating gene-enhancer B_{gene=1,enhancer=1}
experiment_samples = data_subset[data_subset.enhancerID == enhancerID]
N_experiment = experiment_samples.shape[0]

y_control = control_samples.expression.values
y_experiment = experiment_samples.expression.values
guide = experiment_samples.guideID.values
guide = guide - min(guide) + 1

In [118]:
data_subset

,geneID,enhancerID,guideID,cellID,expression
0,1,-1,-1,1,25763
1,1,-1,-1,2,19388
2,1,-1,-1,3,17405
3,1,-1,-1,4,29517
4,1,-1,-1,5,17733
...,...,...,...,...,...
775,1,5,25,776,7725
776,1,5,25,777,8098
777,1,5,25,778,5885
778,1,5,25,779,6509


In [115]:
# import data 

# data {
#   int<lower=0> N_control;                  // Number of control samples
#   int<lower=0> N_experiment;               // Number of experimental samples
#   array[N_control] real y_control;         // Expression levels in control samples
#   array[N_experiment] real y_experiment;   // Expression levels in experimental samples
#   array[N_experiment] int<lower=1> guide; // Guide index for each experiment sample (1 to 5)
# }

data = {'N_control': N_control,
        'N_experiment': N_experiment,
        'y_control': y_control,
        'y_experiment': y_experiment,
        'guide': guide ,
        }


In [ ]:
model_fit = model.sample(data=data,thin=1,iter_sampling=1000,iter_warmup=50,seed=11051999)


In [117]:
model_fit.summary().Mean.to_dict()

{'lp__': -1571.45,
 'mu_control': 20905.2,
 'B': 0.887274,
 'sigma_control': 4537.78,
 'sigma_experiment': 3780.78,
 'potency[1]': 0.63098,
 'potency[2]': 0.17178,
 'potency[3]': 0.462607,
 'potency[4]': 0.578569,
 'potency[5]': 0.487273,
 'mu_experiment': 18531.1304}